In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm

In [ ]:
df = pd.read_excel("/content/drive/MyDrive/AutomatedQuestionGeneration/train.xlsx")

In [ ]:
df = df.drop("Unnamed: 0",axis = 1)

In [ ]:
df.head()

,index,question,context,text,answer_start,c_id
0,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,in the late 1990s,269.0,0
1,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,singing and dancing,207.0,0
2,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,2003,526.0,0
3,56bf6b0f3aeaaa14008c9601,In what city and state did Beyonce grow up?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Houston, Texas",166.0,0
4,56bf6b0f3aeaaa14008c9602,In which decade did Beyonce become famous?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,late 1990s,276.0,0


In [ ]:
!pip3 install bertopic[all]

In [ ]:
from bertopic import BERTopic

In [ ]:
unique_contexts = df["context"].unique()

In [ ]:
topic_model = BERTopic(language="english", calculate_probabilities=True) # We need the probabilities to visualize
topics, _ = topic_model.fit_transform(unique_contexts)

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


In [ ]:
# Get the most frequent topics
topic_freq = topic_model.get_topic_freq()
outliers = topic_freq['Count'][topic_freq['Topic']==-1].iloc[0]
print(f"{outliers} documents have not been classified")
print(f"The other {topic_freq['Count'].sum() - outliers} documents are {topic_freq['Topic'].shape[0]-1} topics")

4266 documents have not been classified
The other 14763 documents are 395 topics


In [ ]:
for i in range(10):
  topic_no = topic_freq['Topic'].iloc[i]
  print("Topic ", topic_no," : \n")
  print(topic_model.get_topic(topic_no))
  print("\n\n")

In [ ]:
# Returns how spatially separated each topic is.
topic_model.visualize_topics()

In [ ]:
import re

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
def Sort_Tuple(tup):
  # Sorting a list of tuples using the second item  
    tup.sort(key = lambda x: x[1]) 
    return tup 

split_ratio = 0.6

def reference_bifurcate_tm(original_context,answer_start,index):  # ... 
  # Total sentences in the original_context
  #n_sentences = original_context.count('. ') + 1
  original_context += " "
  #p = re.compile('\w\. ')
  p = re.compile("[\w|\"|\']\. ")
  n_sentences = len(p.findall(original_context)) 
  # Number of sentenecs to consider in the reference context --> Half the size of the given context
  window_size = int(np.round(n_sentences*split_ratio))
  #Perform NER on each sentence and select window_size number of sentences (which have the least number of Named Entities)
  #context_list = original_context.split(". ")
  #d = ". "
  #context_list =  [e+d for e in original_context.split(d) if e]
  context_list = re.split(p,original_context)
  l = p.findall(original_context)
  l.append("")
  context_list = [a+b for a,b in zip(re.split(p,original_context),l)]
  context_len_list = [len(x) for x in context_list]
  c = 0
  answer_sentence_index = 0
  for elem in context_len_list:
    c += elem
    if c >= answer_start:
      break
    answer_sentence_index += 1


  # # Initialising NER list counter
  # ner_count = []


  # for i in range(n_sentences):
  #   # Appending the number of entities of each sentence to a list
  #   ner_count.append((i,len(nlp(context_list[i]).ents)))
  
  # ner_count = Sort_Tuple(ner_count)

  
  j = 0
  topic_similarity_average = []
  ref_context = ""
  given_context = ""

  if (index % 50) == 0:
    given_context += context_list[answer_sentence_index] 
    window_size += 1
  else:
    ref_context += context_list[answer_sentence_index]  # Answer sentence
    window_size -= 1

  answer_topic = topic_model.find_topics(context_list[answer_sentence_index])

  for j in range(n_sentences):
    if (context_list[j] == context_list[answer_sentence_index]):
      continue
    # if (j < window_size):
    #   ref_context = ". ".join([ref_context,context_list[ner_count[j][0]]])
    # else:
    #   given_context = ". ".join([given_context,context_list[ner_count[j][0]]])
    else:
      temp = topic_model.find_topics(context_list[j])
      # Finding the matching topics to compute the corresponding similarity average
      temp_similarty_list = [key for key, val in enumerate(temp[0]) if val in set(answer_topic[0])]
      s = 0
      for item in temp_similarty_list:
        s += temp[1][item]
      if len(temp_similarty_list) == 0:
        s = 0
      else:  
        s /= len(temp_similarty_list)
      topic_similarity_average.append((j,s))

  topic_similarity_average = Sort_Tuple(topic_similarity_average)

  for k in range(1,window_size + 1):
    # if k == 1:
    #   ref_context = context_list[topic_similarity_average[-k][0]]
    # else:
    ref_context = "".join([ref_context,context_list[topic_similarity_average[-k][0]]])
  for y in range(window_size + 1, n_sentences):
    # if y == window_size + 1:
    #   given_context =  context_list[topic_similarity_average[-y][0]]
    # else:
    given_context = "".join([given_context,context_list[topic_similarity_average[-y][0]]])

  return given_context, ref_context

In [ ]:
p = re.compile('\w\. ')
context_list = re.split(p,df.iloc[3,2])
l = p.findall(df.iloc[3,2])
l.append("")
context_list_temp = [a+b for a,b in zip(re.split(p,df.iloc[3,2]),l)]
context_len_list = [len(x) for x in context_list]

In [ ]:
re.split(p,"I am a disco dancer. ")

['I am a disco dance', '']

In [ ]:
df.iloc[3,2]

'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".'

In [ ]:
temp[temp["answer_start"] == "None"]
temp["answer_start"].unique()

array([ 269.,  207.,  526.,  166.,  276.,  320.,  505.,  360.,  290.,
        590.,  369.,  565.,  260.,  586.,  180.,  406.,   48.,   95.,
        466.,  104.,  935.,  985.,  736.,   18.,  970.,  393.,  445.,
        552.,  303.,  204.,  330.,  578.,  152.,  101.,  255.,  540.,
        117.,   49.,  165.,  507.,  148.,  711.,  484.,  385.,  355.,
        542.,  918.,  537., 1264.,    3.,    7.,  192., 1126.,  215.,
        848., 1212.,   51.,   85.,  849.,  688.,  169.,  714.,  194.,
        396.,  110.,  149.,   37.,  216.,  348.,  793.,  557.,  593.,
        115.,  378.,  628., 1070.,   84.,  331.,  431.,  705.,  834.,
        210.,  416.,  435.,  545.,   58.,  240.,  123.,  193.,  419.,
        474.,  130., 1042.,  229.,  696.,  513.,  664., 1206.,   38.,
        100.,  253.,  132.,  346.,  323.,  342.,   28.,  635.,   53.,
        171.,  550.,  671.,  112.,  576.,  932.,  436.,  487.,   34.,
        897., 1567., 1881.,   78., 1744.,  156.,  605., 1611.,   69.,
        439.,  582.,

In [ ]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   index              200 non-null    object 
 1   question           200 non-null    object 
 2   context            200 non-null    object 
 3   text               200 non-null    object 
 4   answer_start       200 non-null    float64
 5   c_id               200 non-null    int64  
 6   given_context      200 non-null    object 
 7   reference_context  200 non-null    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 12.6+ KB


In [ ]:
temp = df[:500]
temp.head(100)

,index,question,context,text,answer_start,c_id
0,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,in the late 1990s,269.0,0
1,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,singing and dancing,207.0,0
2,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,2003,526.0,0
3,56bf6b0f3aeaaa14008c9601,In what city and state did Beyonce grow up?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Houston, Texas",166.0,0
4,56bf6b0f3aeaaa14008c9602,In which decade did Beyonce become famous?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,late 1990s,276.0,0
...,...,...,...,...,...,...
95,56d462f82ccc5a1400d8311f,Who replaced Luckett and Roberson in Destiny's...,LeToya Luckett and Roberson became unhappy wit...,Farrah Franklin and Michelle Williams.,110.0,7
96,56d462f82ccc5a1400d83120,Who was blamed for Luckett and Roberson leavin...,LeToya Luckett and Roberson became unhappy wit...,Beyoncé,149.0,7
97,56d462f82ccc5a1400d83121,Who helped Beyoncé overcome her depression dur...,LeToya Luckett and Roberson became unhappy wit...,her mother,714.0,7
98,56d462f82ccc5a1400d83123,Which newest member was removed from Destiny's...,LeToya Luckett and Roberson became unhappy wit...,Farrah Franklin,110.0,7


In [ ]:
original_context = df.iloc[0,2] + " "
original_context

'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy". '

In [ ]:
original_context[-1]

' '

In [ ]:
c1 = df.iloc[0,2]
n_sentences = len(p.findall(c1)) 
print(n_sentences)

3


In [ ]:
c1 = df.iloc[0,2] + " "
n_sentences = len(p.findall(c1)) 
print(n_sentences)

3


In [ ]:
"".join(context_list) == original_context

True

In [ ]:
p = re.compile('\w\. ')
n_sentences = len(p.findall(original_context)) 
print(n_sentences)
# Number of sentenecs to consider in the reference context --> Half the size of the given context
#window_size = int(np.round(n_sentences*split_ratio))
#Perform NER on each sentence and select window_size number of sentences (which have the least number of Named Entities)
#context_list = original_context.split(". ")
#d = ". "
#context_list =  [e+d for e in original_context.split(d) if e]
context_list = re.split(p,original_context)
l = p.findall(original_context)
l.append("")
context_list = [a+b for a,b in zip(re.split(p,original_context),l)]
context_len_list = [len(x) for x in context_list]

3


In [ ]:
p = re.compile("[\w|\"|\']\. ")
text = "abc. def. gef\". "
p.findall(text)

['c. ', 'f. ', '". ']

In [ ]:
print(text)

abc. def. gef". 


In [ ]:
context_list

['Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. ',
 "Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. ",
 "Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. ",
 'Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy". ']

In [ ]:
temp["given_context"] = "a"
temp["reference_context"] = "a"

In [ ]:
for i in range(len(temp)):
  temp["given_context"][i],temp["reference_context"][i] = reference_bifurcate_tm(temp["context"][i],temp["answer_start"][i],i)
temp["given_context"] = temp["given_context"]
temp["reference_context"] = temp["reference_context"]

In [ ]:
%debug

In [ ]:
df.iloc[550,2]

'The Bey Hive is the name given to Beyoncé\'s fan base. Fans were previously titled "The Beyontourage", (a portmanteau of Beyoncé and entourage). The name Bey Hive derives from the word beehive, purposely misspelled to resemble her first name, and was penned by fans after petitions on the online social networking service Twitter and online news reports during competitions.'

In [ ]:
 a = reference_bifurcate_tm(temp["context"][140],temp["answer_start"][140],140)

In [ ]:
df.loc[140]

index                                    56be91b23aeaaa14008c90f2
question        Destiny's Child got a star on the Hollywood Wa...
context         In November 2003, she embarked on the Dangerou...
text                                                         2006
answer_start                                                 1212
c_id                                                           11
Name: 140, dtype: object

In [ ]:
df.iloc[140,2]

'In November 2003, she embarked on the Dangerously in Love Tour in Europe and later toured alongside Missy Elliott and Alicia Keys for the Verizon Ladies First Tour in North America. On February 1, 2004, Beyoncé performed the American national anthem at Super Bowl XXXVIII, at the Reliant Stadium in Houston, Texas. After the release of Dangerously in Love, Beyoncé had planned to produce a follow-up album using several of the left-over tracks. However, this was put on hold so she could concentrate on recording Destiny Fulfilled, the final studio album by Destiny\'s Child. Released on November 15, 2004, in the US and peaking at number two on the Billboard 200, Destiny Fulfilled included the singles "Lose My Breath" and "Soldier", which reached the top five on the Billboard Hot 100 chart. Destiny\'s Child embarked on a worldwide concert tour, Destiny Fulfilled... and Lovin\' It and during the last stop of their European tour, in Barcelona on June 11, 2005, Rowland announced that Destiny\'s

In [ ]:
d = ". "
s =  [e+d for e in df.iloc[140,2].split(d) if e]

In [ ]:
s

['In November 2003, she embarked on the Dangerously in Love Tour in Europe and later toured alongside Missy Elliott and Alicia Keys for the Verizon Ladies First Tour in North America. ',
 'On February 1, 2004, Beyoncé performed the American national anthem at Super Bowl XXXVIII, at the Reliant Stadium in Houston, Texas. ',
 'After the release of Dangerously in Love, Beyoncé had planned to produce a follow-up album using several of the left-over tracks. ',
 "However, this was put on hold so she could concentrate on recording Destiny Fulfilled, the final studio album by Destiny's Child. ",
 'Released on November 15, 2004, in the US and peaking at number two on the Billboard 200, Destiny Fulfilled included the singles "Lose My Breath" and "Soldier", which reached the top five on the Billboard Hot 100 chart. ',
 "Destiny's Child embarked on a worldwide concert tour, Destiny Fulfilled... ",
 "and Lovin' It and during the last stop of their European tour, in Barcelona on June 11, 2005, Rowla

In [ ]:
d = ". "
context_list =  [e+d for e in df.iloc[140,2].split(d) if e]
context_len_list = [len(x) for x in context_list]

In [ ]:
context_len_list

[182, 133, 130, 130, 219, 75, 193, 157]

In [ ]:
print(temp.iloc[140,2])

In November 2003, she embarked on the Dangerously in Love Tour in Europe and later toured alongside Missy Elliott and Alicia Keys for the Verizon Ladies First Tour in North America. On February 1, 2004, Beyoncé performed the American national anthem at Super Bowl XXXVIII, at the Reliant Stadium in Houston, Texas. After the release of Dangerously in Love, Beyoncé had planned to produce a follow-up album using several of the left-over tracks. However, this was put on hold so she could concentrate on recording Destiny Fulfilled, the final studio album by Destiny's Child. Released on November 15, 2004, in the US and peaking at number two on the Billboard 200, Destiny Fulfilled included the singles "Lose My Breath" and "Soldier", which reached the top five on the Billboard Hot 100 chart. Destiny's Child embarked on a worldwide concert tour, Destiny Fulfilled... and Lovin' It and during the last stop of their European tour, in Barcelona on June 11, 2005, Rowland announced that Destiny's Chil

In [ ]:
for sentence in context_list:
  print(sentence)

In November 2003, she embarked on the Dangerously in Love Tour in Europe and later toured alongside Missy Elliott and Alicia Keys for the Verizon Ladies First Tour in North America
On February 1, 2004, Beyoncé performed the American national anthem at Super Bowl XXXVIII, at the Reliant Stadium in Houston, Texas
After the release of Dangerously in Love, Beyoncé had planned to produce a follow-up album using several of the left-over tracks
However, this was put on hold so she could concentrate on recording Destiny Fulfilled, the final studio album by Destiny's Child
Released on November 15, 2004, in the US and peaking at number two on the Billboard 200, Destiny Fulfilled included the singles "Lose My Breath" and "Soldier", which reached the top five on the Billboard Hot 100 chart
Destiny's Child embarked on a worldwide concert tour, Destiny Fulfilled..
and Lovin' It and during the last stop of their European tour, in Barcelona on June 11, 2005, Rowland announced that Destiny's Child woul

In [ ]:
len(temp.iloc[140,2])

1217

In [ ]:
answer_sentence_index = 0
c = 0
for i in context_len_list:
  c += i
  if c >= 1212:
    break
  answer_sentence_index += 1

In [ ]:
%debug

> <ipython-input-114-817c1012606a>(45)reference_bifurcate_tm()
     43     window_size += 1
     44   else:
---> 45     ref_context += context_list[answer_sentence_index] # Answer sentence
     46     window_size -= 1
     47 

ipdb> i
140
ipdb> exit


In [ ]:
temp["context"][155]

'Beyoncé\'s second solo album B\'Day was released on September 5, 2006, in the US, to coincide with her twenty-fifth birthday. It sold 541,000 copies in its first week and debuted atop the Billboard 200, becoming Beyoncé\'s second consecutive number-one album in the United States. The album\'s lead single "Déjà Vu", featuring Jay Z, reached the top five on the Billboard Hot 100 chart. The second international single "Irreplaceable" was a commercial success worldwide, reaching number one in Australia, Hungary, Ireland, New Zealand and the United States. B\'Day also produced three other singles; "Ring the Alarm", "Get Me Bodied", and "Green Light" (released in the United Kingdom only).'

In [ ]:
a[0]

'. B\'Day also produced three other singles; "Ring the Alarm", "Get Me Bodied", and "Green Light" (released in the United Kingdom only).. The second international single "Irreplaceable" was a commercial success worldwide, reaching number one in Australia, Hungary, Ireland, New Zealand and the United States'

In [ ]:
a[1]

'The album\'s lead single "Déjà Vu", featuring Jay Z, reached the top five on the Billboard Hot 100 chart. It sold 541,000 copies in its first week and debuted atop the Billboard 200, becoming Beyoncé\'s second consecutive number-one album in the United States. Beyoncé\'s second solo album B\'Day was released on September 5, 2006, in the US, to coincide with her twenty-fifth birthday'

In [ ]:
context_list = temp.iloc[155,2].split(". ")

In [ ]:
context_list = temp.iloc[155,2].split(". ")
context_len_list = [len(x) for x in context_list]

In [ ]:
context_len_list

[122, 152, 103, 169, 132]

In [ ]:
temp.head(500)

,index,question,context,text,answer_start,c_id,given_context,reference_context
0,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,in the late 1990s,269.0,0,"Born and raised in Houston, Texas, she perform...",Their hiatus saw the release of Beyoncé's debu...
1,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,singing and dancing,207.0,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Born and raised in Houston, Texas, she perform..."
2,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,2003,526.0,0,"Born and raised in Houston, Texas, she perform...",Their hiatus saw the release of Beyoncé's debu...
3,56bf6b0f3aeaaa14008c9601,In what city and state did Beyonce grow up?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Houston, Texas",166.0,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Born and raised in Houston, Texas, she perform..."
4,56bf6b0f3aeaaa14008c9602,In which decade did Beyonce become famous?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,late 1990s,276.0,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Born and raised in Houston, Texas, she perform..."
...,...,...,...,...,...,...,...,...
495,56d4e17f2ccc5a1400d832e4,What are Beyoncé's backup singers called?,"In 2006, Beyoncé introduced her all-female tou...",The Mamas,216.0,41,"In 2006, Beyoncé introduced her all-female tou...","Her background singers, The Mamas, consist of ..."
496,56d4e17f2ccc5a1400d832e5,When did The Mamas make their debut?,"In 2006, Beyoncé introduced her all-female tou...",the 2006 BET Awards,343.0,41,The band have supported Beyoncé in most subseq...,They made their debut appearance at the 2006 B...
497,56bec3303aeaaa14008c9391,What characteristics has Beyonce received accl...,Beyoncé has received praise for her stage pres...,stage presence and voice,36.0,42,Jim Farber of the Daily News and Stephanie Cla...,Beyoncé has received praise for her stage pres...
498,56bec3303aeaaa14008c9393,Which former president of Def Jam called Beyon...,Beyoncé has received praise for her stage pres...,L.A. Reid,445.0,42,Jim Farber of the Daily News and Stephanie Cla...,According to Barbara Ellen of The Guardian Bey...


In [ ]:
temp.iloc[0,2]

'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".'

In [ ]:
temp.iloc[0,1]

'When did Beyonce start becoming popular?'

In [ ]:
# Example 1: Given Context

In [ ]:
temp.iloc[0,6]

"Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. "

In [ ]:
# Exmaple 1: Reference Context

In [ ]:
temp.iloc[0,7]

'Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy". Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. '

In [ ]:
temp.to_csv("/content/drive/MyDrive/AutomatedQuestionGeneration/Topic-Model-Bifurcate.csv")

In [ ]:
temp = pd.read_csv("/content/drive/MyDrive/AutomatedQuestionGeneration/Topic-Model-Bifurcate.csv").drop("Unnamed: 0",axis = 1)

In [ ]:
# Example 2: Given Context

In [ ]:
temp.iloc[30,6]

"Throughout a career spanning 19 years, she has sold over 118 million records as a solo artist, and a further 60 million with Destiny's Child, making her one of the best-selling music artists of all time. The Recording Industry Association of America recognized her as the Top Certified Artist in America during the 2000s decade. Time listed her among the 100 most influential people in the world in 2013 and 2014. In 2009, Billboard named her the Top Radio Songs Artist of the Decade, the Top Female Artist of the 2000s and their Artist of the Millennium in 2011"

In [ ]:
# Exmaple 2: Reference Context

In [ ]:
temp.iloc[30,7]

'On stage, her dynamic, highly choreographed performances have led to critics hailing her as one of the best entertainers in contemporary popular music. A self-described "modern-day feminist", Beyoncé creates songs that are often characterized by themes of love, relationships, and monogamy, as well as female sexuality and empowerment. She has won 20 Grammy Awards and is the most nominated woman in the award\'s history. Forbes magazine also listed her as the most powerful female musician of 2015.'